# Update and Query Embeddings
We update embeddings in the original index (core)

In [ ]:
import pysolr
from sentence_transformers import SentenceTransformer
from IPython.display import display,HTML

Create a Solr instance (set autocommit to True (ok for test purpose)

In [ ]:
solr = pysolr.Solr('http://localhost:8983/solr/DenseIndex', always_commit=True)

Load the model

In [ ]:
model = SentenceTransformer("Snowflake/snowflake-arctic-embed-xs")

## Update the Core 
Add embeddings to the existing core

Define a function to embed any text

In [ ]:
def generate_text_embeddings(text):
    embedding = model.encode([text])  # the encode function requires an array
    return embedding[0]  # the encode function returns an array

Define a function to embed a query

In [ ]:
def generate_query_embeddings(query):
    embedding = model.encode([query], prompt_name="query")
    return embedding[0]

Define a function to update the embedding-field in the Solr core.

1. all records are selected
2. records are updated one a the time adding the embedding of the description-fiele

In [ ]:
def update_description_vector():

    #retrieve all the recrods from collection
    solr_response=solr.search(
                        q='*:*',
                        rows=30,
                        start=0,
                        fl='year, description',
                        wt='json')

    for item in solr_response:
        description = item["description"].lower()
        year = item['year']  # year is the unique id - needed to identify to document to be updated
       
        embedding = generate_text_embeddings(description)
        solr.add({'year':year, 'description_vector':{'set':[float(w) for w in embedding]}})
        solr.commit()

Execute the update

In [ ]:
update_description_vector()

## Execute a semantic search

Define a query string

In [ ]:
query = "FIFA World Cup events with a toxic atmosphere"

In [ ]:
query_embedded = generate_query_embeddings(query)

Execute the query

In [ ]:
solr_response=solr.search(fl=['title', 'description', 'score'],
    q="{!knn f=description_vector topK=3}"+str([float(w) for w in query_embedded]),
    rows = 30)


Display the results

In [ ]:
print(f"Found {len(solr_response)} results")
print("Query: "+query)

for doc in solr_response:
    display(HTML("<h3>"+doc['title']+"   Score="+str(doc['score'])+"</h3>"))
    display(HTML(doc['description']))
    print("")